In [ ]:
#load dataset
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# import package
!pip install transformers
!pip install bert-tensorflow 
!pip install pyenchant
!pip install -U pip setuptools wheel
!pip install -U spacy
!sudo apt-get install libenchant1c2a
!python -m spacy download en_core_web_sm
# install the spacy package
!pip install git+https://github.com/MartinoMensio/spacy-universal-sentence-encoder.git
!python -m spacy download en_core_web_lg

import warnings
warnings.filterwarnings('ignore')

import numpy as np 
from numpy import asarray
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns

import sklearn
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import mean_squared_error
from sklearn.feature_extraction.text import CountVectorizer

import nltk
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords
from nltk import word_tokenize
from nltk.tokenize import RegexpTokenizer

import re
import time
import torch
import transformers as ppb

from nltk.corpus import stopwords
import nltk.tokenize as tk
import torch
import torch.nn as nn
import transformers
from transformers import BertTokenizer, BertModel, AutoModel, BertTokenizerFast,BertForSequenceClassification

# Keras functional API
from keras.models import Sequential, Model, load_model, model_from_config
from keras.layers import Dense, Dropout, Embedding, LSTM, Input, Flatten, Lambda
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import keras.backend as K

import os
import csv
import copy

import tensorflow as tf
import tensorflow_hub as hub

#import enchant #  Enchant spellchecking library
import spacy
import re
import urllib.request
from bs4 import BeautifulSoup

# xgboost
!pip install xgboost
import xgboost
from xgboost import XGBRegressor

# cosine similarity
import spacy
nlp = spacy.load('en_core_web_lg')

     |████████████████████████████████| 2.6 MB 8.2 MB/s 
     |████████████████████████████████| 636 kB 57.5 MB/s 
     |████████████████████████████████| 3.3 MB 48.4 MB/s 
     |████████████████████████████████| 895 kB 49.4 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 64 kB 2.8 MB/s 
     |████████████████████████████████| 55 kB 3.4 MB/s 
     |████████████████████████████████| 1.6 MB 7.5 MB/s 
     |████████████████████████████████| 819 kB 39.2 MB/s 
  Attempting uninstall: setuptools
    Found existing installation: setuptools 57.2.0
    Uninstalling setuptools-57.2.0:
      Successfully uninstalled setuptools-57.2.0
  Attempting uninstall: pip
    Found existing installation: pip 21.1.3
    Uninstalling pip-21.1.3:
      Successfully uninstalled pip-21.1.3
ERROR: pip's dependency resolver does not currently take into account all the p

distutils: /usr/local/lib/python3.7/dist-packages
sysconfig: /usr/lib/python3.7/site-packages
distutils: /usr/local/lib/python3.7/dist-packages
sysconfig: /usr/lib/python3.7/site-packages
distutils: /usr/local/include/python3.7/UNKNOWN
sysconfig: /usr/include/python3.7m/UNKNOWN
distutils: /usr/local/bin
sysconfig: /usr/bin
distutils: /usr/local
sysconfig: /usr
user = False
home = None
root = None
prefix = None
     |████████████████████████████████| 6.4 MB 7.6 MB/s 
     |████████████████████████████████| 42 kB 1.5 MB/s 
     |████████████████████████████████| 456 kB 58.1 MB/s 
     |████████████████████████████████| 10.1 MB 37.0 MB/s 
     |████████████████████████████████| 621 kB 59.2 MB/s 
  Attempting uninstall: catalogue
    Found existing installation: catalogue 1.0.0
    Uninstalling catalogue-1.0.0:
      Successfully uninstalled catalogue-1.0.0
  distutils: /usr/local/include/python3.7/catalogue
  sysconfig: /usr/include/python3.7m/catalogue
  distutils: /usr/local/include/pyt

In [ ]:
train = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/asap-aes/Data/training_set_rel3.tsv",sep='\t', encoding='ISO-8859-1')
dev = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/asap-aes/Data/valid_set.tsv",sep='\t', encoding='ISO-8859-1')
test = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/asap-aes/Data/test_set.tsv",sep='\t', encoding='ISO-8859-1')

In [ ]:
prompt = pd.DataFrame({"essay_set":[1,2,3,4,5,6,7,8],
                       "prompt":["More and more people use computers, but not everyone agrees that this benefits society. Those who support advances in technology believe that computers have a positive effect on people. They teach hand-eye coordination, give people the ability to learn about faraway places and people, and even allow people to talk online with other people. Others have different ideas. Some experts are concerned that people are spending too much time on their computers and less time exercising, enjoying nature, and interacting with family and friends. Write a letter to your local newspaper in which you state your opinion on the effects computers have on people. Persuade the readers to agree with you.",
                                "Write a persuasive essay to a newspaper reflecting your vies on censorship in libraries. Do you believe that certain materials, such as books, music, movies, magazines, etc., should be removed from the shelves if they are found offensive? Support your position with convincing arguments from your own experience, observations, and/or reading.",
                                "Write a response that explains how the features of the setting affect the cyclist. In your response, include examples from the essay that support your conclusion.",
                                "Write a response that explains why the author concludes the story with this paragraph. In your response, include details and examples from the story that support your ideas.",
                                "Describe the mood created by the author in the memoir. Support your answer with relevant and specific information from the memoir.",
                                "Based on the excerpt, describe the obstacles the builders of the Empire State Building faced in attempting to allow dirigibles to dock there. Support your answer with relevant and specific information from the excerpt.",
                                "Write about patience. Being patient means that you are understanding and tolerant. A patient person experience difficulties without complaining.Do only one of the following: write a story about a time when you were patient OR write a story about a time when someone you know was patient OR write a story in your own way about patience.",
                                "We all understand the benefits of laughter. For example, someone once said, “Laughter is the shortest distance between two people.” Many other people believe that laughter is an important part of any relationship. Tell a true story in which laughter was one element or part."]})

## Data processing

In [ ]:
def preprocess(df):
  df['normalized_score'] = df['domain1_score'] / df.groupby('essay_set')['domain1_score'].transform('max')

preprocess(train)

In [ ]:
def clean_anonymization(essay):
  '''
  function to remove the anoymaization
  '''
  res=[]
  # Lowercase
  essay = essay.lower()

  # Remove single letter words
  essay = ' '.join( [w for w in essay.split() if len(w)>1] )
  for i in essay.split():
    if i.startswith("@"):
      continue
    else:
      res.append(i)
  return ' '.join(res)

train['essay']=train['essay'].apply(lambda x:clean_anonymization(x))
# also remove from dev and test
dev['essay']=dev['essay'].apply(lambda x:clean_anonymization(x))
test['essay']=test['essay'].apply(lambda x:clean_anonymization(x))

In [ ]:
def remove_stopwords(essay):
    essay = nlp(essay.lower()) 
    result = [] 
    for word in essay: 
        if word.text in nlp.Defaults.stop_words: 
            continue
        result.append(word.text)#5
    return " ".join(result) 

In [ ]:
train_p = train.merge(prompt,on='essay_set',how='left')
train_p = train_p[['essay','normalized_score','prompt']]

In [ ]:
train_p['essay_cleaned'] = train_p['essay'].apply(lambda x:remove_stopwords(x))
train_p['prompt_cleaned'] = train_p['prompt'].apply(lambda x:remove_stopwords(x))

## Prompt Relevant Score

### Cosine similarity


In [ ]:
train_p['essay'][0]

"dear local newspaper, think effects computers have on people are great learning skills/affects because they give us time to chat with friends/new people, helps us learn about the globe(astronomy) and keeps us out of troble! thing about! dont you think so? how would you feel if your teenager is always on the phone with friends! do you ever time to chat with your friends or buisness partner about things. well now there's new way to chat the computer, theirs plenty of sites on the internet to do so: facebook, myspace ect. just think now while your setting up meeting with your boss on the computer, your teenager is having fun on the phone not rushing to get off cause you want to use it. how did you learn about other countrys/states outside of yours? well have by computer/internet, it's new way to learn about what going on in our time! you might think your child spends lot of time on the computer, but ask them so question about the economy, sea floor spreading or even about the you'll be s

In [ ]:
train_p.head()

,essay,normalized_score,prompt,essay_cleaned,prompt_cleaned
0,"dear local newspaper, think effects computers ...",0.666667,"More and more people use computers, but not ev...","dear local newspaper , think effects computers...","people use computers , agrees benefits society..."
1,dear believe that using computers will benefit...,0.750000,"More and more people use computers, but not ev...",dear believe computers benefit ways like talki...,"people use computers , agrees benefits society..."
2,"dear, more and more people use computers, but ...",0.583333,"More and more people use computers, but not ev...","dear , people use computers , agrees benefits ...","people use computers , agrees benefits society..."
3,"dear local newspaper, have found that many exp...",0.833333,"More and more people use computers, but not ev...","dear local newspaper , found experts computers...","people use computers , agrees benefits society..."
4,dear know having computers has positive effect...,0.666667,"More and more people use computers, but not ev...",dear know having computers positive effect peo...,"people use computers , agrees benefits society..."


In [ ]:
similarity_list  = []
for i in range(len(train_p)):
  essay = nlp(train_p['essay_cleaned'][i])
  prompt = nlp(train_p['prompt_cleaned'][i])
  similarity_score =  essay.similarity(prompt)
  similarity_list.append(similarity_score)

In [ ]:
Cosine_similarity = pd.DataFrame(similarity_list,columns= ['Cosine Similarity'])

In [ ]:
# save to google drive for further training
Cosine_similarity.to_csv('Cosine_similarity.csv')
!cp Cosine_similarity.csv "drive/My Drive/Colab Notebooks/asap-aes/"

In [ ]:
pd.concat([Cosine_similarity,train[['normalized_score']]],axis=1).sort_values(by=['Cosine Similarity'],ascending = False)

,Cosine Similarity,normalized_score
402,0.983700,0.750000
2,0.981550,0.583333
1574,0.980989,0.666667
681,0.980943,0.666667
1340,0.979822,0.666667
...,...,...
6444,0.385222,0.333333
6112,0.360957,0.333333
18,0.355787,0.333333
7397,0.347311,0.250000


## Handcrafted Features

In [ ]:
def get_correct_and_incorrect_spelling(df):

  # Load spaCy model
  nlp = spacy.load('en_core_web_sm')
  # load the enchant dictionary
  d = enchant.Dict("en_US")
  
  incorrect_spelling = []
  correct_spelling = []
  
  for essay in df['essay']:
    individual = []
    for word in essay.split():
      individual.append(d.check(word))                                                                         
    incorrect_spelling.append(individual.count(False))
    correct_spelling.append(individual.count(True))
  return incorrect_spelling, correct_spelling


def get_lexical_diversity(df):
    """
    Function that measures lexical diversity which is
    The ratio of total words to unique words
    """
    diversity = []
    for essay in df['essay']:
      diversity.append(round(len(tk.word_tokenize(essay)) / float(len(set(tk.word_tokenize(essay)))), 2))
    return diversity


def get_list_of_number_of_pos(df):
    noun_cnt = []
    verb_cnt= []
    adv_cnt = []
    adj_cnt = []
    
    nlp = spacy.load('en_core_web_sm')
    for essay in df['essay']:
        parsed_essay = nlp(essay)
        token_pos = [token.pos_ for token in parsed_essay]
        noun_cnt.append(token_pos.count('NOUN'))
        verb_cnt.append(token_pos.count('VERB'))
        adv_cnt.append(token_pos.count('ADV'))
        adj_cnt.append(token_pos.count('ADJ'))   
    return noun_cnt, verb_cnt, adv_cnt,adj_cnt

In [ ]:
def get_features(X):
    incorrect_spelling = get_correct_and_incorrect_spelling(X)[0]
    correct_spelling = get_correct_and_incorrect_spelling(X)[1]
    lexical_diversity = get_lexical_diversity(X)
    noun_cnt = get_list_of_number_of_pos(X)[0]
    verb_cnt = get_list_of_number_of_pos(X)[1]
    adv_cnt = get_list_of_number_of_pos(X)[2]
    adj_cnt = get_list_of_number_of_pos(X)[3]
    
    handcrafted_features = pd.DataFrame(
    {'incorrect_spelling': incorrect_spelling,
     'correct_spelling': correct_spelling,
     'lexical_diversity': lexical_diversity,
     'noun_cnt': noun_cnt,
     'verb_cnt': verb_cnt,
     'adv_cnt': adv_cnt,
     'adj_cnt': adj_cnt
    })
    return handcrafted_features

In [ ]:
handcrafted_features_train = get_features(train_p)

In [ ]:
# save to google drive for further training
handcrafted_features_train.to_csv('handcrafted_features_train.csv')
!cp handcrafted_features_train.csv "drive/My Drive/Colab Notebooks/asap-aes/"